
# From Tokens → Embeddings → Attention (Q·K)  
**Sentence:** *"Your journey starts with one step."*

This notebook quickly shows:
1) **Tokenized corpus** vs **embeddings** (why we need vectors)  
2) **Latent dimensions** intuition (toy semantic axes) + **similarity** (cosine & dot product)  
3) **Self-attention** on the same sentence: Q, K, V → attention weights → context vectors


In [ ]:

import math
import torch
import torch.nn as nn
torch.set_printoptions(precision=4, sci_mode=False)
torch.manual_seed(0)



## 1) Tokenized Corpus vs Embeddings

- **Tokenized corpus**: a sequence of discrete symbols (tokens/IDs).  
- **Embeddings**: continuous vectors that *compress* token meaning into numbers (latent features).

We'll use the sentence used in the book: **"Your journey starts with one step."**


In [ ]:

# Tokenization
sentence = "Your journey starts with one step."
tokens = [t.strip(".,!?").lower() for t in sentence.split()]
tokens


In [ ]:

# Build tiny vocab from the sentence
special = ["<pad>", "<bos>", "<eos>"]
word_set = special + sorted(set(tokens))
vocab = {w:i for i,w in enumerate(word_set)}
ivocab = {i:w for w,i in vocab.items()}

def encode(ws):
    return torch.tensor([vocab["<bos>"]] + [vocab[w] for w in ws] + [vocab["<eos>"]], dtype=torch.long)

ids = encode(tokens)
print("Vocab:", vocab)
print("Token IDs:", ids.tolist())



**Takeaway:** token IDs are **discrete**; neural nets need **vectors** to compute.  
Next, we embed each token ID into a **dense vector**. We'll use 3D to keep things readable.


In [ ]:

# 3D embeddings + fixed values for the six words
emb_dim = 3
emb = nn.Embedding(len(vocab), emb_dim)

with torch.no_grad():
    emb.weight.zero_()
    emb.weight[vocab["your"]]    = torch.tensor([0.43, 0.15, 0.89])
    emb.weight[vocab["journey"]] = torch.tensor([0.55, 0.87, 0.66])
    emb.weight[vocab["starts"]]  = torch.tensor([0.57, 0.85, 0.64])
    emb.weight[vocab["with"]]    = torch.tensor([0.22, 0.58, 0.33])
    emb.weight[vocab["one"]]     = torch.tensor([0.77, 0.25, 0.10])
    emb.weight[vocab["step"]]    = torch.tensor([0.05, 0.80, 0.55])

X_tokens = torch.stack([emb.weight[vocab[w]] for w in tokens], dim=0)
print("Token embeddings (3D) for the 6 words:\n", X_tokens)
print("Shape:", X_tokens.shape)



## 2) Latent Dimensions & Similarity (Cosine / Dot)

An embedding dimension can be interpreted as a **latent feature**.  
For intuition, imagine toy axes like:
- dim0: "is_movement"  
- dim1: "is_abstract"  
- dim2: "is_objectness"

> These are made-up for teaching; real models learn their own axes.

We measure **similarity** with **cosine** or **dot product**.  
The attention score uses a **dot product** (scaled) between **Q** and **K**, which is like asking:  
> *Do my interests (Q) align with your attributes (K)?*


In [ ]:

def cosine(a,b, dim=-1, eps=1e-8):
    an = a / (a.norm(dim=dim, keepdim=True)+eps)
    bn = b / (b.norm(dim=dim, keepdim=True)+eps)
    return (an*bn).sum(dim=dim)

pairs = [("journey","starts"), ("one","step"), ("your","journey"), ("with","your")]
for a,b in pairs:
    va, vb = emb.weight[vocab[a]], emb.weight[vocab[b]]
    cos = float(cosine(va, vb))
    dot = float(va @ vb)
    print(f"{a:>7s} vs {b:<7s}  cosine={cos:.3f}  dot={dot:.3f}")



## 3) Add Positional Embeddings

Self-attention alone has no sense of order, so we add positional vectors and sum with token embeddings.


In [ ]:

pos_emb = nn.Embedding(16, 3)
with torch.no_grad():
    pos_emb.weight.copy_(torch.tensor([
        [0.00, 0.00, 0.00],
        [0.01, 0.02, 0.03],
        [0.02, 0.01, -0.01],
        [0.03, 0.00, 0.01],
        [0.04, -0.01, 0.02],
        [0.05, 0.02, 0.00],
        [0.06, 0.03, -0.02],
        [0.07, 0.01, 0.01],
        [0.08, 0.00, -0.01],
        [0.09, -0.02, 0.02],
        [0.10, 0.03, 0.00],
        [0.11, 0.00, 0.01],
        [0.12, 0.01, -0.02],
        [0.13, -0.01, 0.02],
        [0.14, 0.02, 0.00],
        [0.15, 0.01, 0.01],
    ], dtype=torch.float))

pos = torch.arange(len(tokens))
X = X_tokens + pos_emb(pos)
print("X = token + positional embeddings:\n", X)



## 4) Attention on the Sentence (Single Head)

We compute **Q**, **K**, **V**, then:
\[
\text{scores} = \frac{QK^\top}{\sqrt{d_k}}, \quad
\text{weights} = \text{softmax}(\text{scores}), \quad
Z = \text{weights}\cdot V
\]


In [ ]:

head_dim = 2
W_Q = nn.Linear(3, head_dim, bias=False)
W_K = nn.Linear(3, head_dim, bias=False)
W_V = nn.Linear(3, head_dim, bias=False)

with torch.no_grad():
    W_Q.weight.copy_(torch.tensor([[0.5, 0.0, 0.5],
                                   [0.0, 0.5, -0.5]]))
    W_K.weight.copy_(torch.tensor([[0.4, -0.1, 0.3],
                                   [-0.2, 0.6, 0.1]]))
    W_V.weight.copy_(torch.tensor([[0.3, 0.1, -0.2],
                                   [0.1, -0.3, 0.4]]))

Q, K, V = W_Q(X), W_K(X), W_V(X)
scores = (Q @ K.T) / math.sqrt(head_dim)
weights = torch.softmax(scores, dim=-1)
Z = weights @ V

print("Q:\n", Q, "\n")
print("K:\n", K, "\n")
print("V:\n", V, "\n")
print("Scores (scaled QK^T):\n", scores, "\n")
print("Attention weights (rows sum to 1):\n", weights, "\n")
print("Context vectors Z:\n", Z)



## 5) Wrap-up

- **Tokenized corpus**: discrete IDs for words/subwords.  
- **Embeddings**: continuous vectors where dimensions act as **latent features**.  
- **Similarity** via cosine/dot reflects **alignment** of features.  
- **Attention** uses the dot product of \(Q\) and \(K\) so each token “listens” to the most relevant others; 
  the resulting **context vector \(Z\)** is a **weighted mixture** of **V** across the sentence.
